# Example case of simulating observations using a user input file

## Authors: Charles Romero and Emily Moravec

Last updated: 2024-12-30.

## Usage
This notebook shows how to use _M2_ProposalTools_ to simulate observations of a target given a user input FITS file that contains the expected Compton-y map of their object. This notebook is based on the "Example case of simulating observations of a cluster with multiple pointings" example on M2_ProposalTools (https://m2-tj.readthedocs.io/en/latest/Leightweight_SimObs_A10.html which can be downloaded at https://github.com/CharlesERomero/M2_TJ/blob/master/docs/source/Leightweight_SimObs_A10.ipynb). 

In [1]:
### Useful libraries
import numpy as np
import astropy.units as u
from astropy.io import fits 
#### Modules within our library
import M2_ProposalTools.WorkHorse as WH
import M2_ProposalTools.MakeRMSmap as MRM
import M2_ProposalTools.ModelFitting as MF

Let's define some options for the input cluster, mapping parameters, and scanning strategy:

Assume the scan strategy is taken to be a set of four scans which are offset by the respective distance, in arcminutes, from the central pointing towards: (1) the west, (2) north, (3) east, and (4) south.

In [1]:
outdir   = "/USER_SPECIFIED DIRECTORY/" # output directory for products
times    = [22]                 # Time in hours
ptgs     = [[180,45.0]] # Centroid (RA,Dec) in degrees - function is expecting a list of pairs.
sizes    = [3.5]               # Scan sizes, in arcminutes
offsets  = [1.5]                 # Pointing offsets, in arcminutes.

Read in your Compton-y FITS file here.

In [5]:
SkyHDU  = fits.open(<USER_SPECIFIED_FILE>)
pixsize_hdu = WH.get_pixarcsec(SkyHDU)
SkyHDU[0].data = WH.smooth_by_M2_beam(image=SkyHDU[0].data,pixsize=pixsize_hdu)
ShyHDU[0].data *= -3.5 # now in K. Can convert to uK later.

Your input image is what MUSTANG-2 would see if data processing imparted no filtering effects. Unfortunately, it does, and so we combined the filtering effects with the sensitivity mapmaking in the following routine. It returns two HDULists. The first HDUList has a filtered map and a corresponding weight map, where the weight map is the inverse variance. The second HDUList is the same, but the filtered map has an additional smoothing. This smoothing is the standard smoothing applied to MIDAS maps when producing SNR maps; it is with this smoothing that the inverse variance maps, as calculated, are appropriate comparisons.

In [6]:
FilterHDU,SmoothHDU = WH.lightweight_simobs_hdu(SkyHDU,ptgs=ptgs,sizes=sizes,times=times,offsets=offsets,center=None)

Depending on what you want to track, this adds some information to the filenames

In [7]:
pixstr = "{:.1f}".format(pixsize).replace(".","p")
zstr   = "{:.1f}".format(z).replace(".","z")
Mstr   = "{:.1f}".format(M5_14).replace(".","m")
sss    = ["{:.1f}".format(mysz).replace(".","s") for mysz in sizes]
sts    = ["{:.1f}".format(mytime).replace(".","h") for mytime in times]
ssstr  = "_".join(sss)
ststr  = "_".join(sts)
InputStr = "_".join([zstr,Mstr,ssstr,ststr,pixstr])

Save to fits files for inspection.

In [8]:
Skyname = "SimulatedSky_"+InputStr+".fits"
SkyHDU.writeto(outdir+Skyname,overwrite=True)

filename = "SimulatedObs_Unsmoothed_"+InputStr+".fits"
FilterHDU.writeto(outdir+filename,overwrite=True) # should be a list where first entry is signal map with no noise added and second entry is RMS map
filename2 = "SimulatedObs_Smoothed_"+InputStr+".fits"
SmoothHDU.writeto(outdir+filename2,overwrite=True)

If you'd like to add noise.

In [3]:
WhiteNoiseSTD1   = np.random.normal(size=SmoothHDU[0].data.shape) # Standard deviation (RMS) = 1
NoiseRealization = WhiteNoiseSTD1 * SmoothHDU[1].data  # Second term is the actual (MUSTANG-2) RMS map
filename = "SimulatedObs_Smoothed_PlusNoise"+InputStr+".fits"
NoiseRealization.writeto(outdir+filename,overwrite=True)

NameError: name 'np' is not defined